In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder
from pyspark.ml.regression import RandomForestRegressor, rA
spark = SparkSession.builder.getOrCreate()


22/06/08 15:43:59 WARN Utils: Your hostname, arch resolves to a loopback address: 127.0.1.1; using 10.50.61.122 instead (on interface wlp4s0)
22/06/08 15:43:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/08 15:44:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/08 15:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/08 15:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [43]:
carDF = spark.read.option("header", "true").option("delimiter", ",")\
    .option("inferSchema", "true").csv("../datasets/auto-mpg.data.txt")



In [44]:
carDF = carDF.drop("name")
carDF.count()

398

In [45]:
carDF.dropna(how="any")
carDF.count()


398

In [46]:

encoder = OneHotEncoder(inputCol="origin",outputCol="origin_indexed")
carDF = encoder.fit(carDF).transform(carDF)
carDF = carDF.drop("origin")
carDF.show()


+----+---------+------------+----------+------+------------+----------+--------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin_indexed|
+----+---------+------------+----------+------+------------+----------+--------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70| (3,[1],[1.0])|
|15.0|        8|       350.0|       165|  3693|        11.5|        70| (3,[1],[1.0])|
|18.0|        8|       318.0|       150|  3436|        11.0|        70| (3,[1],[1.0])|
|16.0|        8|       304.0|       150|  3433|        12.0|        70| (3,[1],[1.0])|
|17.0|        8|       302.0|       140|  3449|        10.5|        70| (3,[1],[1.0])|
|15.0|        8|       429.0|       198|  4341|        10.0|        70| (3,[1],[1.0])|
|14.0|        8|       454.0|       220|  4354|         9.0|        70| (3,[1],[1.0])|
|14.0|        8|       440.0|       215|  4312|         8.5|        70| (3,[1],[1.0])|
|14.0|        8|       455.0|       225|  4

In [49]:
for each in carDF.columns[:-1]:
    carDF = carDF.withColumn(each + "_", carDF[each].cast("float"))
    carDF = carDF.drop(each)


carDF.show()

+--------------+----+----------+-------------+-----------+-------+-------------+-----------+
|origin_indexed|mpg_|cylinders_|displacement_|horsepower_|weight_|acceleration_|model_year_|
+--------------+----+----------+-------------+-----------+-------+-------------+-----------+
| (3,[1],[1.0])|18.0|       8.0|        307.0|      130.0| 3504.0|         12.0|       70.0|
| (3,[1],[1.0])|15.0|       8.0|        350.0|      165.0| 3693.0|         11.5|       70.0|
| (3,[1],[1.0])|18.0|       8.0|        318.0|      150.0| 3436.0|         11.0|       70.0|
| (3,[1],[1.0])|16.0|       8.0|        304.0|      150.0| 3433.0|         12.0|       70.0|
| (3,[1],[1.0])|17.0|       8.0|        302.0|      140.0| 3449.0|         10.5|       70.0|
| (3,[1],[1.0])|15.0|       8.0|        429.0|      198.0| 4341.0|         10.0|       70.0|
| (3,[1],[1.0])|14.0|       8.0|        454.0|      220.0| 4354.0|          9.0|       70.0|
| (3,[1],[1.0])|14.0|       8.0|        440.0|      215.0| 4312.0|    

In [50]:
carDF.columns[:-1]

['origin_indexed',
 'mpg_',
 'cylinders_',
 'displacement_',
 'horsepower_',
 'weight_',
 'acceleration_']

In [51]:
vec = VectorAssembler(inputCols=['origin_indexed',
 'cylinders_',
 'displacement_',
 'horsepower_',
 'weight_',
 'acceleration_'], outputCol="features")
carDF = vec.setHandleInvalid("keep").transform(carDF)

carDF.show()

+--------------+----+----------+-------------+-----------+-------+-------------+-----------+--------------------+
|origin_indexed|mpg_|cylinders_|displacement_|horsepower_|weight_|acceleration_|model_year_|            features|
+--------------+----+----------+-------------+-----------+-------+-------------+-----------+--------------------+
| (3,[1],[1.0])|18.0|       8.0|        307.0|      130.0| 3504.0|         12.0|       70.0|[0.0,1.0,0.0,8.0,...|
| (3,[1],[1.0])|15.0|       8.0|        350.0|      165.0| 3693.0|         11.5|       70.0|[0.0,1.0,0.0,8.0,...|
| (3,[1],[1.0])|18.0|       8.0|        318.0|      150.0| 3436.0|         11.0|       70.0|[0.0,1.0,0.0,8.0,...|
| (3,[1],[1.0])|16.0|       8.0|        304.0|      150.0| 3433.0|         12.0|       70.0|[0.0,1.0,0.0,8.0,...|
| (3,[1],[1.0])|17.0|       8.0|        302.0|      140.0| 3449.0|         10.5|       70.0|[0.0,1.0,0.0,8.0,...|
| (3,[1],[1.0])|15.0|       8.0|        429.0|      198.0| 4341.0|         10.0|       7

In [52]:
trainDF, testDF = carDF.randomSplit([0.75,0.25],seed=1234)

In [53]:
rf = RandomForestRegressor(featuresCol="features", labelCol="mpg_")
evaluator = RegressionEvaluator(
    labelCol="mpg_", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

rfModel = rf.fit(trainDF)

+--------------+----+----------+-------------+-----------+-------+-------------+-----------+--------------------+
|origin_indexed|mpg_|cylinders_|displacement_|horsepower_|weight_|acceleration_|model_year_|            features|
+--------------+----+----------+-------------+-----------+-------+-------------+-----------+--------------------+
|     (3,[],[])|18.0|       3.0|         70.0|       90.0| 2124.0|         13.5|       73.0|[0.0,0.0,0.0,3.0,...|
|     (3,[],[])|19.0|       6.0|        156.0|      108.0| 2930.0|         15.5|       76.0|[0.0,0.0,0.0,6.0,...|
|     (3,[],[])|20.0|       4.0|         97.0|       88.0| 2279.0|         19.0|       73.0|[0.0,0.0,0.0,4.0,...|
|     (3,[],[])|21.1|       4.0|        134.0|       95.0| 2515.0|         14.8|       78.0|[0.0,0.0,0.0,4.0,...|
|     (3,[],[])|21.5|       3.0|         80.0|      110.0| 2720.0|         13.5|       77.0|[0.0,0.0,0.0,3.0,...|
|     (3,[],[])|22.0|       4.0|        108.0|       94.0| 2379.0|         16.5|       7